<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/150_Agent_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Agent Code

In [ ]:
from typing import Dict, List, Any
import json

class Agent:
    """Base class for all agents"""
    def __init__(self, name: str):
        self.name = name

    def execute(self, task: str, context: Dict = None) -> Dict:
        """Override this method in specific agents"""
        raise NotImplementedError

class ResearchAgent(Agent):
    """Simple research agent"""
    def execute(self, task: str, context: Dict = None) -> Dict:
        # Simulate research work
        return {
            "agent": self.name,
            "result": f"Research completed for: {task}",
            "data": {"findings": ["fact1", "fact2", "fact3"]},
            "status": "success"
        }

class WriterAgent(Agent):
    """Simple writing agent"""
    def execute(self, task: str, context: Dict = None) -> Dict:
        # Use context from previous agents if available
        research_data = context.get("research_data", []) if context else []
        return {
            "agent": self.name,
            "result": f"Article written about: {task}",
            "data": {"article": f"Based on research {research_data}, here's the article..."},
            "status": "success"
        }

class BasicOrchestrator:
    """The simplest possible orchestrator"""

    def __init__(self):
        # 1. AGENT REGISTRY - catalog of available agents
        self.agents: Dict[str, Agent] = {}

        # 2. EXECUTION CONTEXT - shared state between agents
        self.context: Dict[str, Any] = {}

    def register_agent(self, agent: Agent):
        """Add an agent to our toolshed"""
        self.agents[agent.name] = agent
        print(f"Registered agent: {agent.name}")

    def execute_workflow(self, workflow: List[Dict]) -> List[Dict]:
        """
        3. WORKFLOW EXECUTION - the core orchestration logic

        workflow format: [
            {"agent": "research", "task": "Find info about AI"},
            {"agent": "writer", "task": "Write article about AI"}
        ]
        """
        results = []

        for step in workflow:
            agent_name = step["agent"]
            task = step["task"]

            # Get the agent from our registry
            if agent_name not in self.agents:
                results.append({
                    "error": f"Agent '{agent_name}' not found",
                    "status": "failed"
                })
                break

            agent = self.agents[agent_name]

            # Execute the agent with current context
            try:
                result = agent.execute(task, self.context)
                results.append(result)

                # 4. CONTEXT MANAGEMENT - update shared state
                # Pass results to next agents
                if result["status"] == "success":
                    if "data" in result:
                        key = f"{agent_name}_data"
                        self.context[key] = result["data"]

                print(f"✓ {agent_name}: {result['result']}")

            except Exception as e:
                error_result = {
                    "agent": agent_name,
                    "error": str(e),
                    "status": "failed"
                }
                results.append(error_result)
                print(f"✗ {agent_name}: {str(e)}")
                break  # Stop on first failure

        return results

# Example usage
def main():
    # Create orchestrator
    orchestrator = BasicOrchestrator()

    # Register agents (build our toolshed)
    orchestrator.register_agent(ResearchAgent("research"))
    orchestrator.register_agent(WriterAgent("writer"))

    # Define a simple workflow
    workflow = [
        {"agent": "research", "task": "Find information about AI orchestration"},
        {"agent": "writer", "task": "Write an article about AI orchestration"}
    ]

    # Execute workflow
    print("\n--- Executing Workflow ---")
    results = orchestrator.execute_workflow(workflow)

    # Show results
    print("\n--- Results ---")
    for i, result in enumerate(results):
        print(f"Step {i+1}: {json.dumps(result, indent=2)}")

if __name__ == "__main__":
    main()

Let's take a fresh look at our basic orchestrator code and extract the key insights. This simple ~100 line example actually demonstrates all the fundamental principles of intelligent orchestration.

## **🎯 Key Features of Our Basic Orchestrator:**

### **1. The Four Core Components (All Present!)**

```python
# 1. AGENT REGISTRY - catalog of available agents
self.agents: Dict[str, Agent] = {}

# 2. EXECUTION CONTEXT - shared state between agents  
self.context: Dict[str, Any] = {}

# 3. WORKFLOW EXECUTION - core orchestration logic
def execute_workflow(self, workflow: List[Dict]) -> List[Dict]:

# 4. ERROR HANDLING - graceful failure management
except Exception as e:
    # Stop on first failure, return error details
```

### **2. Intelligent Context Flow**
```python
# Context automatically flows between steps
if result["status"] == "success":
    if "data" in result:
        key = f"{agent_name}_data"
        self.context[key] = result["data"]  # Available to next agents
```

**This is the magic!** Each agent's output becomes available to subsequent agents automatically.

### **3. Standardized Agent Interface**
```python
class Agent:
    def execute(self, task: str, context: Dict = None) -> Dict:
        # All agents follow same contract
        return {
            "agent": self.name,
            "result": "what was accomplished",
            "data": {"structured_output": "..."},
            "status": "success"  # or "failed"
        }
```

**Why this matters:** Any agent can be swapped for any other agent - true modularity.

## **🧠 Key Takeaways & Design Principles:**

### **1. Simplicity Enables Complexity**
- **Simple interface** → Complex workflows possible
- **Basic building blocks** → Sophisticated automation
- **Clear contracts** → Reliable composition

### **2. Context as Communication**
```python
# WriterAgent automatically gets ResearchAgent's data
research_data = context.get("research_data", []) if context else []
```
**Insight:** Agents don't need to know about each other - they communicate through shared context.

### **3. Fail-Fast Philosophy**
```python
break  # Stop on first failure
```
**Insight:** When something goes wrong, stop immediately rather than continuing with bad data.

### **4. Declarative Workflow Definition**
```python
workflow = [
    {"agent": "research", "task": "Find information about AI orchestration"},
    {"agent": "writer", "task": "Write an article about AI orchestration"}
]
```
**Insight:** Workflows are data structures, not code. This enables dynamic workflow generation.

## **🚀 What Makes This "Intelligent" Despite Its Simplicity:**

### **1. Dynamic Agent Discovery**
```python
agent = self.agents[agent_name]  # Lookup by name, not hardcoded
```

### **2. Automatic State Management**
```python
self.context[key] = result["data"]  # No manual state passing
```

### **3. Structured Error Handling**
```python
# Consistent error format across all agents
{"error": f"Agent '{agent_name}' not found", "status": "failed"}
```

### **4. Extensible Architecture**
- Add new agents: Just inherit from `Agent` and register
- Change workflows: Just modify the workflow list
- Add capabilities: Extend the base classes

## **🏗️ Scaling Path (What Our Guides Added):**

### **From Basic → Production:**

1. **Agent Registry** → Added metadata, health monitoring, performance tracking
2. **Workflow Execution** → Added parallel execution, retries, conditional logic  
3. **Context Management** → Added scoping, memory management, TTL
4. **Error Handling** → Added circuit breakers, graceful degradation

### **But the Core Remains the Same:**
- Agents register themselves
- Workflows execute step by step
- Context flows automatically
- Errors are handled gracefully

## **🎪 The Beautiful Insight:**

**This basic orchestrator already demonstrates the fundamental transformation:**

```
❌ Traditional Programming:
function1() → manually pass data → function2() → manually pass data → function3()

✅ Orchestrated Intelligence:
Agent1 → [automatic context] → Agent2 → [automatic context] → Agent3
```

## **🎯 Why This Design is Powerful:**

### **1. Composability**
```python
# Same agents, different workflows
workflow_1 = [{"agent": "research", ...}, {"agent": "writer", ...}]
workflow_2 = [{"agent": "writer", ...}, {"agent": "research", ...}]  # Different order!
```

### **2. Reusability**
```python
# Same agent, different contexts
research_agent.execute("AI trends", context_1)
research_agent.execute("Market analysis", context_2)
```

### **3. Testability**
```python
# Each component can be tested independently
test_agent = MockAgent()
orchestrator.register_agent(test_agent)
```

### **4. Observability**
```python
# Complete visibility into what happened
for i, result in enumerate(results):
    print(f"Step {i+1}: {result}")  # Full execution trace
```

## **🔑 The Core Insight:**

**This basic orchestrator proves that intelligent automation doesn't require complex AI models - it requires intelligent architecture.** The LLMs become more effective because the orchestrator handles all the "plumbing."

**Key Principle:** **Simple, well-designed components create emergent intelligence when composed together.**

Your basic orchestrator is actually a **complete intelligent system** - everything else in our guides just makes it production-ready and more sophisticated. But the fundamental intelligence is already there! 🎯



Let's add more agents to demonstrate the modular approach and show how the orchestrator handles increased complexity with minimal changes

## **🎯 Add Complexity - Code Changes Required: ALMOST NONE!**

Here's the beautiful part - let me highlight what changed and what didn't:

### **❌ ZERO Changes to Core Orchestrator Logic:**

```python
# This method is COMPLETELY UNCHANGED:
def execute_workflow(self, workflow: List[Dict]) -> List[Dict]:
    # Same 20 lines of code handle 2 agents or 20 agents!
    for step in workflow:
        agent_name = step["agent"]
        task = step["task"]
        # ... exact same logic ...
```

**The orchestrator doesn't know or care that we added new agents!**

### **✅ What Actually Changed:**

#### **1. Added New Agent Classes (Modular Addition)**
```python
# NEW: Just inherit from Agent base class
class AnalysisAgent(Agent):
class EmailAgent(Agent):
class ValidationAgent(Agent):
```

#### **2. Extended Agent Registration (Same Pattern)**
```python
# SAME pattern, just more agents:
orchestrator.register_agent(ResearchAgent("research"))     # Original
orchestrator.register_agent(AnalysisAgent("analysis"))     # NEW
orchestrator.register_agent(ValidationAgent("validation")) # NEW
orchestrator.register_agent(EmailAgent("email"))           # NEW
```

#### **3. Enhanced Workflow Definition (Same Format)**
```python
# SAME format, just more steps:
workflow = [
    {"agent": "research", "task": "..."},      # Original
    {"agent": "analysis", "task": "..."},      # NEW  
    {"agent": "writer", "task": "..."},        # Original
    {"agent": "validation", "task": "..."},    # NEW
    {"agent": "email", "task": "..."}          # NEW
]
```

## **🧠 Key Insights - Why This Works:**

### **1. The Agent Interface is the Magic**
```python
# Every agent follows the SAME contract:
def execute(self, task: str, context: Dict = None) -> Dict:
    return {
        "agent": self.name,
        "result": "what was accomplished",
        "data": {...},  # Structured output
        "status": "success"  # or "failed"
    }
```

**Because all agents follow the same interface, the orchestrator treats them identically.**

### **2. Context Automatically Scales**
```python
# Same context code now handles 5 agents worth of data:
key = f"{agent_name}_data"
self.context[key] = result["data"]

# Context now contains:
# - research_data
# - analysis_data  
# - writer_data
# - validation_data
# - email_data
```

**The context management scales automatically - no changes needed!**

### **3. Agent Interdependencies Work Automatically**
```python
# ValidationAgent automatically gets data from ALL previous agents:
research_data = context.get("research_data", {})
analysis_data = context.get("analysis_data", {})
writer_data = context.get("writer_data", {})

# EmailAgent summarizes ALL previous work:
if writer_data: email_body += f"Article: {writer_data.get('word_count')} words"
if analysis_data: email_body += f"Risk: {analysis_data.get('risk_score')}"
```

**Agents automatically become aware of each other through shared context!**

## **🚀 What This Demonstrates:**

### **1. True Modularity**
- **Add agents**: Just inherit and register
- **Remove agents**: Just don't register them
- **Swap agents**: Register different implementation with same name
- **Reorder workflow**: Just change the workflow list

### **2. Emergent Intelligence**
```python
# ValidationAgent becomes smarter as more agents contribute context:
if research_confidence >= 0.8: validation_score += 0.3
if analysis_insights >= 2: validation_score += 0.2  
if content_length >= 50: validation_score += 0.1
```

**The validation becomes more sophisticated automatically as workflow complexity increases!**

### **3. Context-Driven Communication**
```python
# EmailAgent doesn't know about specific agents, just looks for context:
writer_data = context.get("writer_data", {})  # Might exist
analysis_data = context.get("analysis_data", {})  # Might exist

# Adapts email content based on what's available!
```

**Agents become loosely coupled but highly coordinated.**

## **🎪 The Beautiful Pattern:**

### **From Simple to Complex with NO Orchestrator Changes:**

```
Simple Workflow:
Research → Writer
(2 agents, 2 context entries)

Complex Workflow:  
Research → Analysis → Writer → Validation → Email
(5 agents, 5 context entries, complex interdependencies)

SAME ORCHESTRATOR CODE! 🤯
```

## **🔑 Key Takeaways:**

### **1. Interface Design is Everything**
The `Agent` base class interface enables unlimited scaling without orchestrator changes.

### **2. Context as Universal Language**
Shared context enables agents to discover and use each other's work automatically.

### **3. Composition over Configuration**
Add complexity by adding agents, not by changing orchestrator logic.

### **4. Emergent Behavior**
Complex workflow behavior emerges from simple agent interactions.

## **🎯 The Profound Insight:**

**Good architecture means adding complexity doesn't require changing existing code.** We went from 2 agents to 5 agents with complex interdependencies, and the orchestrator core didn't change a single line!

This is the power of **interface-driven design** - the orchestrator becomes a universal execution engine that can handle any agents that follow the contract.

The orchestrator has become a **universal intelligence platform**! 🚀

In [ ]:
from typing import Dict, List, Any
import json
import time
import random

class Agent:
    """Base class for all agents"""
    def __init__(self, name: str):
        self.name = name

    def execute(self, task: str, context: Dict = None) -> Dict:
        """Override this method in specific agents"""
        raise NotImplementedError

class ResearchAgent(Agent):
    """Simple research agent"""
    def execute(self, task: str, context: Dict = None) -> Dict:
        # Simulate research work with some processing time
        time.sleep(0.5)  # Simulate work
        return {
            "agent": self.name,
            "result": f"Research completed for: {task}",
            "data": {
                "findings": ["AI adoption is accelerating", "Market size is $50B", "Growth rate is 25% annually"],
                "sources": ["industry_report.pdf", "market_research.com", "expert_interviews"],
                "confidence": 0.85
            },
            "status": "success"
        }

class WriterAgent(Agent):
    """Simple writing agent"""
    def execute(self, task: str, context: Dict = None) -> Dict:
        # Use context from previous agents if available
        research_data = context.get("research_data", {}) if context else {}
        analysis_data = context.get("analysis_data", {}) if context else {}

        # Simulate writing based on available context
        time.sleep(0.3)

        article_content = f"Article about: {task}\n"
        if research_data:
            article_content += f"Based on research findings: {research_data.get('findings', [])}\n"
        if analysis_data:
            article_content += f"Analysis shows: {analysis_data.get('insights', [])}\n"

        return {
            "agent": self.name,
            "result": f"Article written about: {task}",
            "data": {
                "article": article_content,
                "word_count": len(article_content.split()),
                "sections": ["introduction", "main_findings", "conclusion"]
            },
            "status": "success"
        }

# NEW AGENT: Data Analysis Agent
class AnalysisAgent(Agent):
    """Analyzes data and provides insights"""
    def execute(self, task: str, context: Dict = None) -> Dict:
        # Look for research data to analyze
        research_data = context.get("research_data", {}) if context else {}

        time.sleep(0.4)  # Simulate analysis work

        insights = []
        recommendations = []

        if research_data:
            # Analyze research findings
            findings = research_data.get("findings", [])
            confidence = research_data.get("confidence", 0.0)

            if confidence > 0.8:
                insights.append("High confidence research provides solid foundation")
                recommendations.append("Proceed with aggressive market strategy")
            else:
                insights.append("Research confidence is moderate, need more validation")
                recommendations.append("Conduct additional market validation")

            if len(findings) >= 3:
                insights.append("Comprehensive research covers multiple aspects")
                recommendations.append("Develop multi-faceted approach")
        else:
            insights.append("No research data available for analysis")
            recommendations.append("Conduct research before proceeding")

        return {
            "agent": self.name,
            "result": f"Analysis completed for: {task}",
            "data": {
                "insights": insights,
                "recommendations": recommendations,
                "risk_score": random.uniform(0.1, 0.5),  # Simulated risk assessment
                "confidence": 0.9
            },
            "status": "success"
        }

# NEW AGENT: Email Agent
class EmailAgent(Agent):
    """Sends email notifications"""
    def execute(self, task: str, context: Dict = None) -> Dict:
        # Gather context from previous steps
        writer_data = context.get("writer_data", {}) if context else {}
        analysis_data = context.get("analysis_data", {}) if context else {}

        time.sleep(0.2)  # Simulate email sending

        # Build email content from available context
        email_subject = f"Workflow Complete: {task}"
        email_body = "Workflow execution summary:\n\n"

        if writer_data:
            email_body += f"Article created with {writer_data.get('word_count', 0)} words\n"
            email_body += f"Sections: {', '.join(writer_data.get('sections', []))}\n\n"

        if analysis_data:
            email_body += f"Key insights: {', '.join(analysis_data.get('insights', []))}\n"
            email_body += f"Risk score: {analysis_data.get('risk_score', 'N/A')}\n\n"

        email_body += "All workflow steps completed successfully."

        return {
            "agent": self.name,
            "result": f"Email sent: {task}",
            "data": {
                "recipient": "stakeholder@company.com",
                "subject": email_subject,
                "body": email_body,
                "sent_at": time.time(),
                "delivery_status": "delivered"
            },
            "status": "success"
        }

# NEW AGENT: Validation Agent
class ValidationAgent(Agent):
    """Validates work quality and completeness"""
    def execute(self, task: str, context: Dict = None) -> Dict:
        time.sleep(0.3)  # Simulate validation work

        validation_results = []
        overall_score = 0.0
        issues_found = []

        # Validate research quality
        research_data = context.get("research_data", {}) if context else {}
        if research_data:
            confidence = research_data.get("confidence", 0.0)
            sources = research_data.get("sources", [])

            if confidence >= 0.8:
                validation_results.append("Research confidence meets quality threshold")
                overall_score += 0.3
            else:
                issues_found.append("Research confidence below threshold (0.8)")

            if len(sources) >= 3:
                validation_results.append("Sufficient research sources provided")
                overall_score += 0.2
            else:
                issues_found.append("Insufficient research sources (minimum 3)")
        else:
            issues_found.append("No research data found for validation")

        # Validate analysis quality
        analysis_data = context.get("analysis_data", {}) if context else {}
        if analysis_data:
            insights = analysis_data.get("insights", [])
            recommendations = analysis_data.get("recommendations", [])

            if len(insights) >= 2:
                validation_results.append("Analysis provides sufficient insights")
                overall_score += 0.2
            else:
                issues_found.append("Analysis lacks sufficient insights")

            if len(recommendations) >= 2:
                validation_results.append("Analysis provides actionable recommendations")
                overall_score += 0.2
            else:
                issues_found.append("Analysis lacks actionable recommendations")
        else:
            issues_found.append("No analysis data found for validation")

        # Validate content quality
        writer_data = context.get("writer_data", {}) if context else {}
        if writer_data:
            word_count = writer_data.get("word_count", 0)
            sections = writer_data.get("sections", [])

            if word_count >= 50:  # Minimum content length
                validation_results.append("Content meets minimum length requirement")
                overall_score += 0.1
            else:
                issues_found.append("Content below minimum length requirement")
        else:
            issues_found.append("No content found for validation")

        # Determine overall validation status
        validation_passed = overall_score >= 0.7 and len(issues_found) == 0

        return {
            "agent": self.name,
            "result": f"Validation completed for: {task}",
            "data": {
                "validation_passed": validation_passed,
                "overall_score": overall_score,
                "validation_results": validation_results,
                "issues_found": issues_found,
                "quality_rating": "high" if overall_score >= 0.8 else "medium" if overall_score >= 0.6 else "low"
            },
            "status": "success" if validation_passed else "failed"
        }

class BasicOrchestrator:
    """The simplest possible orchestrator - UNCHANGED CORE LOGIC!"""

    def __init__(self):
        # 1. AGENT REGISTRY - catalog of available agents
        self.agents: Dict[str, Agent] = {}

        # 2. EXECUTION CONTEXT - shared state between agents
        self.context: Dict[str, Any] = {}

    def register_agent(self, agent: Agent):
        """Add an agent to our toolshed - NO CHANGES NEEDED"""
        self.agents[agent.name] = agent
        print(f"Registered agent: {agent.name}")

    def execute_workflow(self, workflow: List[Dict]) -> List[Dict]:
        """
        3. WORKFLOW EXECUTION - CORE LOGIC COMPLETELY UNCHANGED!

        The same orchestrator code handles 2 agents or 20 agents!
        """
        results = []

        for step in workflow:
            agent_name = step["agent"]
            task = step["task"]

            # Get the agent from our registry - SAME CODE
            if agent_name not in self.agents:
                results.append({
                    "error": f"Agent '{agent_name}' not found",
                    "status": "failed"
                })
                break

            agent = self.agents[agent_name]

            # Execute the agent with current context - SAME CODE
            try:
                result = agent.execute(task, self.context)
                results.append(result)

                # 4. CONTEXT MANAGEMENT - SAME CODE HANDLES MORE COMPLEX CONTEXT!
                # Pass results to next agents
                if result["status"] == "success":
                    if "data" in result:
                        key = f"{agent_name}_data"
                        self.context[key] = result["data"]

                print(f"✓ {agent_name}: {result['result']}")

            except Exception as e:
                error_result = {
                    "agent": agent_name,
                    "error": str(e),
                    "status": "failed"
                }
                results.append(error_result)
                print(f"✗ {agent_name}: {str(e)}")
                break  # Stop on first failure

        return results

    # NEW HELPER METHOD: Show current context state
    def show_context(self):
        """Display current context state - NEW but not required for core functionality"""
        print("\n--- Current Context ---")
        for key, value in self.context.items():
            if isinstance(value, dict):
                print(f"{key}: {len(value)} items")
                for subkey in value.keys():
                    print(f"  - {subkey}")
            else:
                print(f"{key}: {type(value).__name__}")

# Example usage showing increased complexity
def main():
    # Create orchestrator - SAME CODE
    orchestrator = BasicOrchestrator()

    # Register agents - JUST MORE OF THE SAME PATTERN!
    orchestrator.register_agent(ResearchAgent("research"))
    orchestrator.register_agent(AnalysisAgent("analysis"))  # NEW
    orchestrator.register_agent(WriterAgent("writer"))
    orchestrator.register_agent(ValidationAgent("validation"))  # NEW
    orchestrator.register_agent(EmailAgent("email"))  # NEW

    # Define a more complex workflow - SAME FORMAT, JUST MORE STEPS!
    workflow = [
        {"agent": "research", "task": "Research AI market trends and opportunities"},
        {"agent": "analysis", "task": "Analyze research findings and provide recommendations"},
        {"agent": "writer", "task": "Write comprehensive market analysis report"},
        {"agent": "validation", "task": "Validate report quality and completeness"},
        {"agent": "email", "task": "Send completion notification to stakeholders"}
    ]

    # Execute workflow - SAME CODE!
    print("\n--- Executing Enhanced Workflow ---")
    results = orchestrator.execute_workflow(workflow)

    # Show context evolution
    orchestrator.show_context()

    # Show results - SAME CODE!
    print("\n--- Results Summary ---")
    for i, result in enumerate(results):
        status_icon = "✅" if result.get("status") == "success" else "❌"
        agent_name = result.get("agent", f"Step {i+1}")
        print(f"{status_icon} {agent_name}: {result.get('result', result.get('error', 'Unknown'))}")

    # NEW: Show data flow between agents
    print("\n--- Data Flow Analysis ---")
    print("Context keys created by each agent:")
    for key in orchestrator.context.keys():
        agent_name = key.replace("_data", "")
        data_size = len(orchestrator.context[key]) if isinstance(orchestrator.context[key], dict) else 1
        print(f"  {agent_name} → {key} ({data_size} data elements)")

if __name__ == "__main__":
    main()